# pYPKa_Z_GIS2 and pYPKa_E_GIS2

This notebook describe the construction of E. coli vectors [pYPKa_Z_GIS2](pYPKa_Z_GIS2.gb) and [pYPKa_E_GIS2](pYPKa_E_GIS2.gb)

The insert defined below is cloned in pYPKa using the blunt restriction
enzymes [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html) and [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) in
two different plasmids. The insert cloned in ZraI will be used as a promoter, while in the EcoRV site it will be used as a
terminator.

![pYPKa_Z and pYPKa_E plasmids](pYPK_ZE.png "pYPKa_Z and pYPKa_E plasmids")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.io/) available online.
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
from pydna.readers import read
from pydna.parsers import parse_primers
from pydna.genbank import Genbank
from pydna.amplify import pcr
from pydna.amplify import Anneal

The vector backbone pYPKa is read from a local [file](pYPKa.gb).

In [2]:
pYPKa = read("pYPKa.gb")

Both restriction enzymes are imported from [Biopython](http://biopython.org/wiki/Main_Page)

In [3]:
from Bio.Restriction import ZraI, EcoRV

The vector is cut with both enzymes.

In [4]:
pYPKa_ZraI  = pYPKa.linearize(ZraI)
pYPKa_EcoRV = pYPKa.linearize(EcoRV)

The template below comes from a Genbank [record](http://www.ncbi.nlm.nih.gov/nuccore/BK006947.3).
Access to Genbank is needed in order to download the template.
If you execute this script, change the email address below to your own.
Always tell Genbank who you are, when using their web service.

In [5]:
gb = Genbank("bjornjobb@gmail.com")

The template is downloaded from Genbank below.

In [6]:
template = gb.nucleotide("BK006947.3 REGION: complement(167791..168042)")

The template is a 252 bp linear DNA fragment.

In [7]:
template

Gbank(BK006947.3  167791-168042)(-252)

The insert has the sequence shown below.

In [8]:
print(str(template.seq))

ACAGGTAGTATATTGCCGTTTGTTGCCTTTATGGCTTAGGGATTTGAGCTTTATAGTAAGAACTGCTGTGGTGTCATGATAGACGAAATGCTGAAAAAATTTTCTTTTCGTTCCTTTTCTCTTTTGGCAAGAGTTGTAATAAGGTCATCGATGGATTAGAAATGCTCTTTCTGCTTTGGGGGCACTTGAGGATTGTATTATTTTAAGGCTTTCCCTTCAGAAATCAGGAAAAAAGAAAAGAATACGAAAAAG


The seguid checksum of the template should be

```9bly8VWuaBxH_5LZvilT13gNxQw```

In [9]:
template.seguid()

9bly8VWuaBxH_5LZvilT13gNxQw

Two primers are used to amplify the insert:

In [10]:
fp,rp = parse_primers(""">1024_fw252
                         ttaaatACAGGTAGTATATTGCCGT
                         >1023_rv252
                         taattaaCTTTTTCGTATTCTTTTCTT""")

PCR to create the insert using the primers above.

In [11]:
prd = pcr(fp, rp, template)
assert str(fp.seq) in prd

The PCR product has this length in bp.

In [12]:
len(prd)

265

A figure of the primers annealing on template.

In [13]:
prd.figure()

      5ACAGGTAGTATATTGCCGT...AAGAAAAGAATACGAAAAAG3
                             |||||||||||||||||||| tm 47.7 (dbd) 53.8
                            3TTCTTTTCTTATGCTTTTTCaattaat5
5ttaaatACAGGTAGTATATTGCCGT3
       ||||||||||||||||||| tm 53.5 (dbd) 55.4
      3TGTCCATCATATAACGGCA...TTCTTTTCTTATGCTTTTTC5

A suggested PCR program.

In [14]:
prd.program()


Taq (rate 30 nt/s) 35 cycles             |265bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 49.5°C/ 0min 8s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 34%
|         |         30s           |      |4-12°C

The final vectors are:

In [15]:
pYPKa_Z_GIS2 = (pYPKa_ZraI  + prd).looped().synced(pYPKa)
pYPKa_E_GIS2 = (pYPKa_EcoRV + prd).looped().synced(pYPKa)

The final vectors with reverse inserts are created below. These vectors theoretically make up
fifty percent of the clones. The PCR strategy below is used to identify clones with the insert
in the correct orientation.

In [16]:
pYPKa_Z_GIS2b = (pYPKa_ZraI  + prd.rc()).looped().synced(pYPKa)
pYPKa_E_GIS2b = (pYPKa_EcoRV + prd.rc()).looped().synced(pYPKa)

A combination of standard primers and the primers above are
used for the strategy to identify correct clones.
Standard primers are listed [here](primers.fasta).

In [17]:
p = { x.id: x for x in parse_primers(""">577
                                         gttctgatcctcgagcatcttaagaattc
                                         >578
                                         gttcttgtctcattgccacattcataagt
                                         >468
                                         gtcgaggaacgccaggttgcccact
                                         >467
                                         ATTTAAatcctgatgcgtttgtctgcacaga
                                         >567
                                         GTcggctgcaggtcactagtgag
                                         >568
                                         GTGCcatctgtgcagacaaacg
                                         >775
                                         gcggccgctgacTTAAAT
                                         >778
                                         ggtaaatccggatTAATTAA
                                         >342
                                         CCTTTTTACGGTTCCTGGCCT""") }

## Diagnostic PCR confirmation

The correct structure of pYPKa_Z_GIS2 is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the GIS2fw primer specific for the insert
in a multiplex PCR reaction with
all three primers present.

Two PCR products are expected if the insert was cloned, the sizes depend
on the orientation. If the vector is empty or contains another insert, only one
product is formed.

#### Expected PCR products sizes from pYPKa_Z_GIS2:

pYPKa_Z_GIS2 with insert in correct orientation.

In [18]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_GIS2).products

[Amplicon(1199), Amplicon(1031)]

pYPKa_Z_GIS2 with insert in reverse orientation.

In [19]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_GIS2b).products

[Amplicon(1199), Amplicon(433)]

Empty pYPKa clone.

In [20]:
Anneal( (p['577'], p['342'], fp), pYPKa).products

[Amplicon(934)]

#### Expected PCR products sizes pYPKa_E_GIS2:

pYPKa_E_GIS2 with insert in correct orientation.

In [21]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_GIS2).products

[Amplicon(1199), Amplicon(950)]

pYPKa_E_GIS2 with insert in reverse orientation.

In [22]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_GIS2b).products


[Amplicon(1199), Amplicon(514)]

Calculate cseguid checksums for the resulting plasmids for future reference.
Cseguid is a checksum that uniquely describes a circular double stranded
sequence.

pYPKa_Z_GIS2.cseguid() should be GFrpptpYRCUg6y0trPnUhX9So0I

In [23]:
print(pYPKa_Z_GIS2.cseguid())
assert pYPKa_Z_GIS2.cseguid() == "GFrpptpYRCUg6y0trPnUhX9So0I"

GFrpptpYRCUg6y0trPnUhX9So0I


pYPKa_E_GIS2.cseguid() should be 5V2RIwqAOqgDJIzn8R6WSYhF9HE

In [24]:
print(pYPKa_E_GIS2.cseguid())
assert pYPKa_E_GIS2.cseguid() == "5V2RIwqAOqgDJIzn8R6WSYhF9HE"

5V2RIwqAOqgDJIzn8R6WSYhF9HE


The sequences are named based on the name of the cloned insert.

In [25]:
pYPKa_Z_GIS2.locus = "pYPKa_Z_GIS2"[:16]
pYPKa_E_GIS2.locus = "pYPKa_Z_GIS2"[:16]

Stamp sequence with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [26]:
pYPKa_Z_GIS2.stamp()
pYPKa_E_GIS2.stamp()

cSEGUID_5V2RIwqAOqgDJIzn8R6WSYhF9HE

Sequences are written to local files.

In [27]:
pYPKa_Z_GIS2.write("pYPKa_Z_GIS2.gb")
pYPKa_E_GIS2.write("pYPKa_E_GIS2.gb")

pYPKa_Z_GIS2.gb

pYPKa_E_GIS2.gb

# [pYPKa_Z_GIS2](pYPKa_Z_GIS2.gb)

In [28]:
from pydna.readers import read
reloaded = read("pYPKa_Z_GIS2.gb")
reloaded.verify_stamp()

cSEGUID_GFrpptpYRCUg6y0trPnUhX9So0I

# [pYPKa_E_GIS2](pYPKa_E_GIS2.gb)

In [29]:
from pydna.readers import read
reloaded = read("pYPKa_E_GIS2.gb")
reloaded.verify_stamp()

cSEGUID_5V2RIwqAOqgDJIzn8R6WSYhF9HE